In [1]:
import config as cfg
import utils.helper_functions as hf
import utils.inputs as inputs
import utils.outputs as outputs
import utils.tf_model as tf_model
import utils.evaluate as eval

from IPython.display import display, clear_output

import pandas as pd
import numpy as np

import hyperopt
from hyperopt import hp, fmin, tpe

import os

os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1' # disable file validation in the debugger
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' #0: All logs (default setting), 1: Filter out INFO logs, up to 3
pd.options.mode.copy_on_write = True # avoid making unnecessary copies of DataFrames or Series


In [2]:
if cfg.use_hyperopt:
    print(cfg.hyperopt_n_iterations)
else:
    num_combinations = hf.get_num_combinations(cfg.param_grid) 

number of combinations: 160


In [3]:
df = pd.read_pickle(cfg.db_path)
df = hf.get_rows_after_date(df, cfg.start_date)
df = hf.fillnavalues(df)

def get_single_level_df(df, ohlcv):
    new_df = df[[ohlcv]]
    new_df = hf.remove_top_column_name(new_df)

    return new_df

def get_ohlcv_dfs(df):
    df_open = get_single_level_df(df, 'Open')
    df_high = get_single_level_df(df, 'High')
    df_low = get_single_level_df(df, 'Low')
    df_close = get_single_level_df(df, 'Close')
    df_volume = get_single_level_df(df, 'Volume')
    
    return {'df_open': df_open, 'df_high': df_high, 'df_low': df_low,
            'df_close': df_close, 'df_volume': df_volume}

num_tickers = hf.get_num_tickers(get_single_level_df(df, 'Open'))
print(f'number of tickers: {num_tickers}')


number of tickers: 593


In [4]:
def get_df_data(hyperparams):
    df_buy = get_single_level_df(df, hyperparams['buying_time'])
    df_sell = get_single_level_df(df, hyperparams['selling_time'])
    dfs_ohlcv = get_ohlcv_dfs(df)

    if os.path.exists(cfg.transformed_data_path) and cfg.use_saved_transformed_data:
        df_data = pd.read_pickle(cfg.transformed_data_path)
        print(f'using existing {cfg.transformed_data_path}')
    else:
        print(f'need to create {cfg.transformed_data_path}')
        df_data = inputs.get_inputs(df_buy, dfs_ohlcv, hyperparams['buying_time'])
        
        df_data.to_pickle(cfg.transformed_data_path)
        print(f'saved new {cfg.transformed_data_path}')

    df_data = outputs.add_outputs(df_data, df_buy, df_sell, dfs_ohlcv, num_tickers, cfg.output_class_name, cfg.fee, **hyperparams)

    df_data = df_data.dropna()

    return df_data

In [5]:
from itertools import product

i = 0
results = []

def objective(hyperparams):
    hyperparams['thresholds'] = [hyperparams['thresholds']]
    hyperparams['rank_pct_thresholds'] = [hyperparams['rank_pct_thresholds']]
    
    df_data = get_df_data(hyperparams)
    test_train_data, model = tf_model.load_model(df_data, hyperparams)
    performance_metrics = eval.evaluate_model(df_data, model, test_train_data, hyperparams)

    result = {**performance_metrics, **hyperparams, 'epochs': cfg.epochs}
    print(result)
    results.append(result)

    performance = result['performance_score']

    return -performance

if cfg.use_hyperopt:
    best = fmin(objective, cfg.search_space, algo=tpe.suggest, max_evals=cfg.hyperopt_n_iterations)
    print(f'best parameters: {best}')
else:
    for params in product(*cfg.param_grid.values()):
        i += 1
        # clear_output(wait=True) # clear printed outputs
        hf.print_combination(i, num_combinations)

        hyperparams = dict(zip(cfg.param_grid.keys(), params))

        df_data = get_df_data(hyperparams)
        test_train_data, model = tf_model.load_tf_model(df_data, hyperparams)
        performance_metrics = eval.evaluate_model(df_data, model, test_train_data, num_tickers, num_combinations, hyperparams)

        result = {**performance_metrics, **hyperparams, 'epochs': cfg.epochs}
        print(result)
        results.append(result)


step: 1/160
need to create ./outputs/classifier_transformed_data.pkl
saved new ./outputs/classifier_transformed_data.pkl
number of elements in y_train: 1605684
number of elements in y_test: 60000
need to create ./outputs/classifier_model.keras


/Users/alexis/.pyenv/versions/3.11.8/lib/python3.11/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


50178/50178 ━━━━━━━━━━━━━━━━━━━━ 49s 952us/step - accuracy: 0.5843 - loss: 1.1400 - val_accuracy: 0.6170 - val_loss: 0.6295
X_test shape: (60000, 55)
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 389us/step
market cumulative % per class: [0.12631667 1.        ]
{'performance_score': 101.7342251521279, 'trimmed_average_profit': 1.0006825246561557, 'average_profit': 1.0170609712381498, 'median_profit': 0.9760878243512974, 'prediction_is_buy_count': 132, 'loss_limit_reached_pct': 0.6515151515151515, 'market_rate': 0.12631666666666666, 'true_positives': 35, 'true_negatives': 52324, 'false_positives': 97, 'false_negatives': 7544, 'winning_rate': 0.26515151515151514, 'winning_rate_vs_market': 0.13883484848484848, 'buying_time': 'Close', 'selling_time': 'Close', 'target_future_days': 1, 'loss_limit': 0.98, 'sell_at_target': False, 'size_layer_1': 128, 'size_layer_2': 128, 'size_layer_3': 128, 'dropout_rate': 0.1, 'balance_data': True, 'batch_size': 32, 'n_first_classes': [0, 0], 'cumulated_probs_target':

/Users/alexis/.pyenv/versions/3.11.8/lib/python3.11/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


50178/50178 ━━━━━━━━━━━━━━━━━━━━ 49s 957us/step - accuracy: 0.5442 - loss: 1.0247 - val_accuracy: 0.5816 - val_loss: 0.6703
X_test shape: (60000, 55)
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 380us/step
market cumulative % per class: [0.2463 1.    ]
{'performance_score': 7.415725968035396, 'trimmed_average_profit': 0.9760878243512974, 'average_profit': 0.9817310755463874, 'median_profit': 0.9760878243512974, 'prediction_is_buy_count': 9, 'loss_limit_reached_pct': 0.7777777777777778, 'market_rate': 0.2463, 'true_positives': 0, 'true_negatives': 45213, 'false_positives': 9, 'false_negatives': 14778, 'winning_rate': 0.0, 'winning_rate_vs_market': -0.2463, 'buying_time': 'Close', 'selling_time': 'Close', 'target_future_days': 1, 'loss_limit': 0.98, 'sell_at_target': False, 'size_layer_1': 128, 'size_layer_2': 128, 'size_layer_3': 128, 'dropout_rate': 0.1, 'balance_data': True, 'batch_size': 32, 'n_first_classes': [0, 0], 'cumulated_probs_target': 0.8, 'thresholds': [1.01], 'rank_pct_thresholds': [

...

50024/50024 ━━━━━━━━━━━━━━━━━━━━ 29s 561us/step - accuracy: 0.5094 - loss: 1.2018 - val_accuracy: 0.7697 - val_loss: 0.6529
X_test shape: (60000, 55)
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 238us/step
market cumulative % per class: [0.1579 1.    ]
{'performance_score': nan, 'trimmed_average_profit': nan, 'average_profit': 0.9971606823898174, 'median_profit': 0.9930199600798403, 'prediction_is_buy_count': 5, 'loss_limit_reached_pct': 0.8, 'market_rate': 0.1579, 'true_positives': 1, 'true_negatives': 50522, 'false_positives': 4, 'false_negatives': 9473, 'winning_rate': 0.2, 'winning_rate_vs_market': 0.0421, 'buying_time': 'Close', 'selling_time': 'Close', 'target_future_days': 10, 'loss_limit': 0.997, 'sell_at_target': False, 'size_layer_1': 128, 'size_layer_2': 128, 'size_layer_3': 128, 'dropout_rate': 0.1, 'balance_data': True, 'batch_size': 32, 'n_first_classes': [0, 0], 'cumulated_probs_target': 0.9, 'thresholds': [1.01], 'rank_pct_thresholds': [0.45], 'epochs': 1}
step: 159/160
need to cr

/Users/alexis/.pyenv/versions/3.11.8/lib/python3.11/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


50024/50024 ━━━━━━━━━━━━━━━━━━━━ 28s 553us/step - accuracy: 0.5164 - loss: 1.2467 - val_accuracy: 0.5062 - val_loss: 0.6874
X_test shape: (60000, 55)
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 239us/step
market cumulative % per class: [0.12485 1.     ]
{'performance_score': nan, 'trimmed_average_profit': nan, 'average_profit': 0.9930199600798403, 'median_profit': 0.9930199600798403, 'prediction_is_buy_count': 1, 'loss_limit_reached_pct': 1.0, 'market_rate': 0.12485, 'true_positives': 0, 'true_negatives': 52508, 'false_positives': 1, 'false_negatives': 7491, 'winning_rate': 0.0, 'winning_rate_vs_market': -0.12485, 'buying_time': 'Close', 'selling_time': 'Close', 'target_future_days': 10, 'loss_limit': 0.997, 'sell_at_target': False, 'size_layer_1': 128, 'size_layer_2': 128, 'size_layer_3': 128, 'dropout_rate': 0.1, 'balance_data': True, 'batch_size': 32, 'n_first_classes': [0, 0], 'cumulated_probs_target': 0.9, 'thresholds': [1.03], 'rank_pct_thresholds': [0.45], 'epochs': 1}
step: 160/160
need 

/Users/alexis/.pyenv/versions/3.11.8/lib/python3.11/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


50024/50024 ━━━━━━━━━━━━━━━━━━━━ 28s 553us/step - accuracy: 0.5258 - loss: 1.2673 - val_accuracy: 0.5800 - val_loss: 0.6738
X_test shape: (60000, 55)
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 240us/step
market cumulative % per class: [0.10623333 1.        ]
{'performance_score': nan, 'trimmed_average_profit': nan, 'average_profit': 0.9930199600798403, 'median_profit': 0.9930199600798403, 'prediction_is_buy_count': 3, 'loss_limit_reached_pct': 1.0, 'market_rate': 0.10623333333333333, 'true_positives': 0, 'true_negatives': 53623, 'false_positives': 3, 'false_negatives': 6374, 'winning_rate': 0.0, 'winning_rate_vs_market': -0.10623333333333333, 'buying_time': 'Close', 'selling_time': 'Close', 'target_future_days': 10, 'loss_limit': 0.997, 'sell_at_target': False, 'size_layer_1': 128, 'size_layer_2': 128, 'size_layer_3': 128, 'dropout_rate': 0.1, 'balance_data': True, 'batch_size': 32, 'n_first_classes': [0, 0], 'cumulated_probs_target': 0.9, 'thresholds': [1.04], 'rank_pct_thresholds': [0.45], 'e

In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df_results = pd.DataFrame(results)
df_results = df_results.sort_values(by='performance_score', ascending=False)
df_results.head(1000)

,performance_score,trimmed_average_profit,average_profit,median_profit,prediction_is_buy_count,loss_limit_reached_pct,market_rate,true_positives,true_negatives,false_positives,false_negatives,winning_rate,winning_rate_vs_market,buying_time,selling_time,target_future_days,loss_limit,sell_at_target,size_layer_1,size_layer_2,size_layer_3,dropout_rate,balance_data,batch_size,n_first_classes,cumulated_probs_target,thresholds,rank_pct_thresholds,epochs
30,119.617552,1.021145,1.043690,0.993020,194,0.623711,0.049300,54,56902,140,2904,0.278351,0.229051,Close,Close,1,0.997,False,128,128,128,0.1,True,32,"[0, 0]",0.9,[1.03],[0.45],1
22,116.343859,1.017609,1.052338,0.990032,126,0.650794,0.051733,36,56806,90,3068,0.285714,0.233981,Close,Close,1,0.994,False,128,128,128,0.1,True,32,"[0, 0]",0.9,[1.03],[0.45],1
24,116.227427,1.017481,1.040534,0.993020,191,0.654450,0.112583,53,53107,138,6702,0.277487,0.164904,Close,Close,1,0.997,False,128,128,128,0.1,True,32,"[0, 0]",0.8,[1.0184],[0.45],1
23,114.282449,1.015337,1.034147,0.990032,235,0.638298,0.028817,52,58088,183,1677,0.221277,0.192460,Close,Close,1,0.994,False,128,128,128,0.1,True,32,"[0, 0]",0.9,[1.04],[0.45],1
14,110.671730,1.011271,1.040015,0.986048,154,0.655844,0.053300,40,56688,114,3158,0.259740,0.206440,Close,Close,1,0.990,False,128,128,128,0.1,True,32,"[0, 0]",0.9,[1.03],[0.45],1
15,110.412630,1.010975,1.024185,0.986048,418,0.607656,0.029500,93,57905,325,1677,0.222488,0.192988,Close,Close,1,0.990,False,128,128,128,0.1,True,32,"[0, 0]",0.9,[1.04],[0.45],1
31,110.306738,1.010853,1.023977,0.993020,426,0.659624,0.027683,77,57990,349,1584,0.180751,0.153068,Close,Close,1,0.997,False,128,128,128,0.1,True,32,"[0, 0]",0.9,[1.04],[0.45],1
26,109.795742,1.010267,1.020860,0.993020,612,0.632353,0.049300,129,56559,483,2829,0.210784,0.161484,Close,Close,1,0.997,False,128,128,128,0.1,True,32,"[0, 0]",0.8,[1.03],[0.45],1
91,109.553214,1.009988,1.049908,0.993020,211,0.829384,0.066733,29,55814,182,3975,0.137441,0.070707,Close,Close,3,0.997,False,128,128,128,0.1,True,32,"[0, 0]",0.8,[1.04],[0.45],1
59,108.637612,1.008929,1.022652,0.993020,590,0.737288,0.050933,99,56453,491,2957,0.167797,0.116863,Close,Close,2,0.997,False,128,128,128,0.1,True,32,"[0, 0]",0.8,[1.04],[0.45],1


In [7]:
df_results.to_excel(f'./outputs/{hf.get_date()}_classifier_results.xlsx')